# Imports 

This Notebook is sequentially processing Images from SuperCaustics to be used in Cleargrasp Training. 
The Code currently does not check for many things that can go wrong during copy-pasting data, so definitely make backups from your images before running this. 

In [ ]:
import numpy as np
import imageio
import PIL.Image as Image
from PIL import ImageFilter
from PIL import ImageChops
import io
import os
from glob import glob
import shutil
import cv2
import random as r

In [ ]:
imageio.plugins.freeimage.download()

# Make Dataset Folders

In [ ]:
#point this to the RAW Screenshots folder. Make Backups before running. You can Comment Out any files that you are not collecting.
os.chdir('E:/')
dir_src = ('E:/SuperCaustics/Saved/Screenshots/Windows/')

Depth =   (dir_src + "Depth/")
Normal = (dir_src + "/Normal/")
Pic =     (dir_src + "/Picture_No_Caustic/")
Caustic = (dir_src + "/Picture_Caustic/")
Caustic_Seg = (dir_src + "/Caustic_Seg/")
Outlines = (dir_src + "/Outlines/")
Outlines_Processed = (dir_src + "/Outlines_Processed/")
Seg = (dir_src + "/Seg/")
Seg_Processed = (dir_src + "/Seg_Processed/")
Depth_exr =   (dir_src + "/Depth_exr/")
Normal_exr = (dir_src + "/Normal_exr/")
Seg_exr = (dir_src + "/Seg_exr/")

try:
    os.mkdir(Depth)
    os.mkdir(Normal)
    os.mkdir(Pic)
    os.mkdir(Caustic_Seg)
    os.mkdir(Caustic)
    os.mkdir(Outlines)
    os.mkdir(Seg)
    os.mkdir(Outlines_Processed)
    os.mkdir(Seg_Processed)
    os.mkdir(Depth_exr)
    os.mkdir(Normal_exr)
    os.mkdir(Seg_exr)

except OSError:
        print('Directory not created.')



# Move images into their respective folders (Order is important)

Use the Same order as you did when capturing your data from AIP SuperCaustics. In my case, I did this in the following order.

In [ ]:
caustic = 1
counter = 0
for filename in os.listdir(dir_src):

    if counter == 0:
        if filename.endswith('.png'):
            shutil.move(dir_src + filename, Caustic) #Caustic
            print("moved " + filename, end='\r')
            counter = counter+1

    elif counter == 1:
        if filename.endswith('.png'):
            shutil.move(dir_src + filename, Pic) #Pic
            print("moved " + filename, end='\r')
            counter = counter+1
            
    elif counter == 2:
        if filename.endswith('.png'):
            shutil.move(dir_src + filename, Depth) #Depth
            print("moved " + filename, end='\r')
            counter = 3
            
    elif counter == 3:
        if filename.endswith('.png'):
            shutil.move(dir_src + filename, Seg) #Seg
            print("moved " + filename, end='\r')
            counter = 0


# Rename raw images in each folder to have the same name

In [ ]:
i = 0
for filename in os.listdir(Pic):
    os.rename(Pic + '/' + filename, Pic + '/' + str(i) + '.png')
    i = i + 1
    print("renamed " + filename, end='\r')
    
v = 0
for filename in os.listdir(Caustic):
    os.rename(Caustic + '/' + filename, Caustic + '/' + str(v) + '.png')
    v = v + 1
    print("renamed " + filename, end='\r')

j = 0
for filename in os.listdir(Depth):
    os.rename(Depth + '/' + filename, Depth + '/' + str(j) + '.png')
    j = j + 1
    print("renamed " + filename, end='\r')

k = 0
for filename in os.listdir(Normal):
    os.rename(Normal + '/' + filename, Normal + '/' + str(k) + '.png')
    k = k + 1
    print("renamed " + filename, end='\r')
    
l = 0
for filename in os.listdir(Outlines):
    os.rename(Outlines + '/' + filename, Outlines + '/' + str(l) + '.png')
    l = l + 1
    print("renamed " + filename, end='\r')
    
m = 0
for filename in os.listdir(Seg):
    os.rename(Seg + '/' + filename, Seg + '/' + str(m) + '.png')
    m = m + 1
    print("renamed " + filename, end='\r')
    
n = 0
for filename in os.listdir(Depth):
    os.rename(Depth + '/' + filename, Depth + '/' + str(n) + '.png')
    n = n + 1
    print("renamed " + filename, end='\r')


# Seg & Outlines should be L.

In [ ]:
for filename in os.listdir(Seg):
    segaddress = Seg + filename
    newaddress = Seg_Processed + filename
    img = []
    arr = np.array(Image.open(segaddress).convert('L'))
    img = np.where(arr>= 220, 255,-1)
    cv2.imwrite(newaddress, img)
    print(filename + 'Processed.', end='\r')
    
print('Seg Files Processed.')

for filename in os.listdir(Outlines):
    outaddress = Outlines + filename
    newaddress = Outlines_Processed + filename
    image = Image.open(outaddress)
    image = image.convert('L')
    arr = np.asarray(image)
    arr = np.floor(arr/126)
    cv2.imwrite(newaddress, arr)
    print(filename + 'Processed.', end='\r')
    
print('Outline Files Processed.')

# Processing Caustic Segmentation

In [ ]:
def CausticGT(image, caustic , seg, thresh = 10):
    import PIL.Image as Image
    #Binary Function
    fn = lambda x : 255 if x > thresh else 0
    
    img = Image.open(image).convert('RGB')
    img_no_caustic = Image.open(caustic).convert('RGB')
    segm = Image.open(seg).convert('1')
    
    #Subtract the Two input Images
    Precursor = ImageChops.subtract(img, img_no_caustic)
    
    #Make Binary Image
    r = Precursor.convert('L').point(fn, mode='1')
    
    #Remove Noise
    p = ImageChops.subtract(r, segm)
    p = p.filter(ImageFilter.MinFilter(5))
    p = p.filter(ImageFilter.MaxFilter(5))
    
    return p

In [ ]:
for filename in os.listdir(Seg_Processed):
    Seg = str(Seg_Processed + filename)
    Image = str(Caustic + filename)
    NoCaustic = str(Pic + filename)
    CausticGT(Image, NoCaustic, Seg).save(Caustic_Seg + filename)
    print(filename + 'Processed.', end='\r')

# Choosing a Random Validation Set: Making the Folders

In [ ]:
dir_val = ('E:/Dataset/Dataset/ExampleData_val/')

valDepth =   (dir_val + "/Depth/")
valNormal = (dir_val + "/Normal/")
valPic =     (dir_val + "/Picture_No_Caustic/")
valCaustic = (dir_val + "/Picture_Caustic/")
valOutlines = (dir_val + "/Outlines/")
valOutlines_Processed = (dir_val + "/Outlines_Processed/")
valSeg = (dir_val + "/Seg/")
valSeg_Processed = (dir_val + "/Seg_Processed/")
valDepth_exr =   (dir_val + "/Depth_exr/")
valNormal_exr = (dir_val + "/Normal_exr/")
valSeg_exr = (dir_val + "/Seg_exr/")

try:
    os.mkdir(dir_val)
    os.mkdir(valDepth)
    os.mkdir(valNormal)
    os.mkdir(valPic)
    os.mkdir(valCaustic)
    os.mkdir(valOutlines)
    os.mkdir(valSeg)
    os.mkdir(valOutlines_Processed)
    os.mkdir(valSeg_Processed)
    os.mkdir(valDepth_exr)
    os.mkdir(valNormal_exr)
    os.mkdir(valSeg_exr)

except OSError:
        print('Directory not created.')



# Moving The Processed Files to Validation Set

In [ ]:
valcount = 0
validationSetSize = 6
for filename in os.listdir(Pic):
    if valcount < validationSetSize and filename.endswith('.png') and r.randint(0,6) == 6:
            
            shutil.move(Depth + filename, valDepth)
            shutil.move(Normal + filename, valNormal)
            shutil.move(Pic + filename, valPic)
            shutil.move(Caustic + filename, valCaustic)
            shutil.move(Outlines + filename, valOutlines)
            shutil.move(Outlines_Processed + filename, valOutlines_Processed)
            shutil.move(Seg + filename, valSeg)
            shutil.move(Seg_Processed + filename, valSeg_Processed)
            shutil.move(Depth_exr + filename.replace(".png", ".exr"), valDepth_exr)
            shutil.move(Normal_exr + filename.replace(".png", ".exr"), valNormal_exr)
            shutil.move(Seg_exr + filename.replace(".png", ".exr"), valSeg_exr)
            
            print("Chosen " + filename, end='\r')
            valcount += 1 